#Steps for exporting pretrained models to ONNX

##Prepare the requirements

###Install ONNX library

In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 39.4 MB/s eta 0:00:00


###Download all the pre-trained models

In [ ]:
!wget http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/squeezeseg.tar.gz
!wget http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/squeezeseg-crf.tar.gz
!wget http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/squeezesegV2.tar.gz
!wget http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/squeezesegV2-crf.tar.gz
!wget http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/darknet21.tar.gz
!wget http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/darknet53.tar.gz
!wget http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/darknet53-1024.tar.gz
!wget http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/darknet53-512.tar.gz


--2025-02-14 17:20:21--  http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/squeezeseg.tar.gz
Resolving www.ipb.uni-bonn.de (www.ipb.uni-bonn.de)... 131.220.233.2
Connecting to www.ipb.uni-bonn.de (www.ipb.uni-bonn.de)|131.220.233.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3395228 (3.2M) [application/x-gzip]
Saving to: ‘squeezeseg.tar.gz’

squeezeseg.tar.gz   100%[===================>]   3.24M  18.8MB/s    in 0.2s    

2025-02-14 17:20:22 (18.8 MB/s) - ‘squeezeseg.tar.gz’ saved [3395228/3395228]

--2025-02-14 17:20:22--  http://www.ipb.uni-bonn.de/html/projects/bonnetal/lidar/semantic/models/squeezeseg-crf.tar.gz
Resolving www.ipb.uni-bonn.de (www.ipb.uni-bonn.de)... 131.220.233.2
Connecting to www.ipb.uni-bonn.de (www.ipb.uni-bonn.de)|131.220.233.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63729995 (61M) [application/x-gzip]
Saving to: ‘squeezeseg-crf.tar.gz’

squeezeseg-crf.tar. 100%[=================

###Extract the pre-trained models zipped files

In [ ]:
import tarfile
import os

root = "/content"
for tar_path in os.listdir(root):
    if tar_path.endswith(".tar.gz"):
        extract_path = tar_path.replace(".tar.gz", "")
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall(path=extract_path)

        print(f"Extracted files to: {extract_path}")


Extracted files to: darknet53
Extracted files to: squeezesegV2-crf
Extracted files to: squeezeseg-crf
Extracted files to: darknet21
Extracted files to: darknet53-512
Extracted files to: squeezeseg
Extracted files to: darknet53-1024
Extracted files to: squeezesegV2


###Clone the repository of the model

In [ ]:
!git clone https://github.com/PRBonn/lidar-bonnetal.git

Cloning into 'lidar-bonnetal'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 109 (delta 0), reused 0 (delta 0), pack-reused 106 (from 1)
Receiving objects: 100% (109/109), 17.44 MiB | 9.98 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [ ]:
%cd /content/lidar-bonnetal/train

/content/lidar-bonnetal/train


##Modify the model archeticture config & make it

### fix booger.TRAIN_PATH error

In [ ]:
import sys
import types

# Create (or override) a dummy module named "__init__" with TRAIN_PATH defined
module_name = "__init__"
train_path = "/content/lidar-bonnetal/train"

# If a module named "__init__" is not already in sys.modules, create one.
if module_name not in sys.modules:
    dummy_module = types.ModuleType(module_name)
    dummy_module.TRAIN_PATH = train_path
    sys.modules[module_name] = dummy_module
else:
    # Otherwise, just set its TRAIN_PATH attribute.
    sys.modules[module_name].TRAIN_PATH = train_path

import __init__ as booger
print("booger.TRAIN_PATH =", booger.TRAIN_PATH)

booger.TRAIN_PATH = /content/lidar-bonnetal/train


###Make the model & export it to ONNX

In [ ]:
import torch
import yaml

from tasks.semantic.postproc.CRF import CRF
from tasks.semantic.modules.segmentator import Segmentator

model_path = "/content/squeezesegV2/squeezesegV2"

with open(model_path + "/arch_cfg.yaml", "r") as f:  # Replace with the correct path
    ARCH = yaml.safe_load(f)

with open(model_path + "/data_cfg.yaml", "r") as f:
    dataset_cfg = yaml.safe_load(f)

# Extract the number of classes from the learning map
learning_map = dataset_cfg["learning_map"]
n_classes = len(set(learning_map.values()))  # Get unique class indices

print(f"Number of classes: {n_classes}")


# Initialize model
model = Segmentator(ARCH, n_classes, path=model_path)

# Load weights (this is done in the model __init__ function if path is given)
model.eval()  # Set to evaluation mode

# Define dummy input tensor (adjust shape as needed)
height = ARCH["dataset"]["sensor"]["img_prop"]["height"]
width = ARCH["dataset"]["sensor"]["img_prop"]["width"]
dummy_input = torch.randn(1, 5, height, width)
dummy_mask = torch.ones(1, height, width)

# Convert to ONNX
onnx_path = "/content/squeezesegV2_segmentator.onnx"
torch.onnx.export(
    model,
    (dummy_input, dummy_mask),
    onnx_path,
    export_params=True,  # Save trained parameters
    opset_version=11,  # Use ONNX opset 11
    do_constant_folding=True,  # Optimize
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "mask": {0: "batch_size"}, "output": {0: "batch_size"}}
)

print(f"Model exported to {onnx_path}")


Number of classes: 20
Using SqueezeNet Backbone
Depth of backbone input =  5
Original OS:  16
New OS:  16
Strides:  [2, 2, 2, 2]
Decoder original OS:  16
Decoder new OS:  16
Decoder strides:  [2, 2, 2, 2]
Total number of parameters:  928464
Total number of parameters requires_grad:  928464
Param encoder  735676
Param decoder  181248
Param head  11540
Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
Model exported to /content/squeezesegV2_segmentator.onnx
